In [1]:
import trimet_plan_game_reset as tpg
import geopandas as gpd

In [2]:
origin_str = "45.582637, -122.754038"
# "45.512908, -122.711987"
destination_str = "45.512894, -122.656701"
#  "45.510896, -122.611478"
planner_response = tpg.call_planner(origin_str, destination_str)

In [3]:
itineraries_df = tpg.get_itinerary_paths(planner_response)
itineraries_df.head(3)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry
0,0,4540,1198,2968,374,1409.119448,0,WALK,WALK,,Origin,3474,N Lombard & Charleston,"LINESTRING (-122.75348 45.58308, -122.75398 45..."
0,0,4540,1198,2968,374,1409.119448,1,75,BUS,3474,N Lombard & Charleston,14176,NE M L King & Russet,"LINESTRING (-122.75177 45.58921, -122.75176 45..."
0,0,4540,1198,2968,374,1409.119448,2,WALK,WALK,14176,NE M L King & Russet,5950,NE M L King & Russet,"LINESTRING (-122.66135 45.57816, -122.66135 45..."


In [4]:
itineraries_reduced, itinerary_routes_reduced = tpg.create_intinerary_gdf_and_reduce(itineraries_df)

In [5]:
itineraries_reduced[itineraries_reduced['itin_idx']==2].explore()